In [1]:
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd

In [2]:
proj_folder = Path('/lustre/orion/lrn057/scratch/yhuang2/rtal/RealTimeAlignment/frontier/')
plot_folder = proj_folder/'plots'
job_roots = proj_folder/'job_roots'
training_folders = list(job_roots.glob('mlp*'))

In [3]:
def string_to_dict(string):
    tokens = string.split('|')[1:]
    params = {}
    for token in tokens:
        key, val = token.split('-')
        if val in ['True', 'False']:
            params[key] = True if val == 'True' else False
        else:
            params[key] = int(val)
    return params

def dict_to_string(dict):
    return 'mlp|' + '|'.join(f'{key}-{val}' for key, val in dict.items())

In [4]:
df = pd.DataFrame(data=[string_to_dict(str(folder)) for folder in training_folders])
sort_by_cols = ['rounded', 'num_features', 'num_subset_solvers', 'subset_size', 'subset_solver_depth']
df = df.sort_values(by=sort_by_cols).reset_index(drop=True)

In [5]:
columns = ['diff_pc', 'diff_ps', 'res_pc', 'res_cc', 'res_sc', 'epsilon']
split = 'train'
last_num_epochs = 10

rows = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    folder = dict_to_string(dict(row))

    perf_series = [row]
    
    for split in ['train', 'valid']:
        csv_fname = job_roots/folder/f'checkpoints/{split}_log.csv'
        csv = pd.read_csv(csv_fname)[columns]
        series = csv[-last_num_epochs:].mean()
        series.rename(index={key: f'{split}_{key}' for key in series.index}, inplace=True)
        perf_series.append(series)
    
    rows.append(pd.concat(perf_series))

df = pd.concat(rows, axis=1).T
df.to_csv(job_roots/'performance.csv', index=False)

  0%|          | 0/600 [00:00<?, ?it/s]